<a href="https://colab.research.google.com/github/JonathanStiefel/Player-Behavior-and-cheating-detection-in-Chess/blob/main/Player_Behavior_and_Cheating_Detection_in_Chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Data

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'chess-com-titled-tuesday-games:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4019214%2F7341679%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240715%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240715T174626Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D22968a2aa2b7bc8ecae61cc045bed59dec4b56d6db039fe79759c680175a846140158a31315d71964c0c8077820717d0317b02d552c225870065476255bbdd2dee80c9ace5dfab00023aa3a921d0c344fb726e93d2a0aa1df269acf201b48e0411bdcf74f6cf6a6eb12fd46f8514e0f57a0ebf1129325ef869d9806119fb978c38e531def66b8d420e689d11990fb8e279734fc5be825d78a97233551599e6bfca4117b87a049b6bd7ebe98a869d8f1da0712694f5bdfaa013853a51d2bce9803dfe0920dec8690264ba34f38e913a98006d9f5dc83c136f054daf7e269c2c5932b1c188bbdcba56b8341a4c261c145f993b4aa80c254e330b1139b44631dcca'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 459598530 bytes downloaded
Downloaded and uncompressed: chess-com-titled-tuesday-games
Data source import complete.


<div style="text-align: right; width: 50%; float: right; font-style: italic;">
There is an old parable according to which Alexander the Great selected people for his army in the following way. A tiger or lion was brought into the same room as the subject. If the subject turned red, it was considered that he had successfully passed the test. If a person turned pale, then such a person was not taken into the army.
</div>

<div style="clear: both;">&nbsp;</div>



Recently Kramnik put a video describing his method for detecting players who might use some external help in decisive games. In short he compares player performance (estimated over large number of tournaments) in rounds 7-10 with their performance in round 11 when competing for money or high places. He discovered that there are some players who in the last 11th round perform >100 points higher than in previous 7-10 rounds, which he finds suspicious.

There are a lot of important nuances when doing such work and also there are risks to fall into some logic traps, paying attention to only data that confirms your hypothesis and discrarding the rest.

In this notebook I'm trying to reproduce his results mostly to understand them better myself and to at least roughly validate his findings.

*Update: at this point is no longer exact replication, but I'm trying to correct a few subtle (but important) details, which I'll explain further, though it could be easily modified to mimic his method closely.*

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/_jk_r8FeREM?si=S2jYU7wtfbP10ST9" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

##Install/Import Libraries

[Russian version](https://youtu.be/BA1LfvYtwbs?si=u9dDpMcU6XofKeAa)

In [ ]:
%%capture
!pip install seaborn pandarallel --upgrade

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)
pd.set_option('display.max_columns', 100)
so.Plot.config.theme.update(mpl.rcParams)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Load Data

In [ ]:
df = pd.read_csv('/kaggle/input/chess-com-titled-tuesday-games/games.csv',
                 low_memory=False, parse_dates=['date', 'start_time'], keep_default_na=False)

In [ ]:
df

,game_id,date,tournament_id,round,start_time,time_control,termination,eco,handle,opponent_handle,name,opponent_name,rating,opponent_rating,result,opponent_result,final_rank,opponent_final_rank,initial_rank,opponent_initial_rank,title,opponent_title,fed,opponent_fed,final_score,opponent_final_score,tie_break,opponent_tie_break,member_since,opponent_member_since,is_blocked,opponent_is_blocked,is_enabled,opponent_is_enabled,color
0,953822338,2014-10-28,-monthly-32-blitz-masters-443681,1,2014-10-28 19:08:10,180+2,won by checkmate,E61,Nouki,ruzomberok,,,2600,2245,1.0,0.0,13,26,1,27,GM,IM,FRA,SWE,5.0,4.0,27.5,11.00,2013-05-14 00:53:05,2008-05-03 19:33:11,0,0,0,1,white
0,953822338,2014-10-28,-monthly-32-blitz-masters-443681,1,2014-10-28 19:08:10,180+2,won by checkmate,E61,ruzomberok,Nouki,,,2245,2600,0.0,1.0,26,13,27,1,IM,GM,SWE,FRA,4.0,5.0,11.0,27.50,2008-05-03 19:33:11,2013-05-14 00:53:05,0,0,1,0,white
1,953822339,2014-10-28,-monthly-32-blitz-masters-443681,1,2014-10-28 19:03:07,180+2,won on time,A00,TomsKantans,lorcho,Toms Kantans,David Larino,2234,2632,0.0,1.0,47,17,28,2,GM,GM,LVA,ESP,0.0,5.0,0.0,16.50,2009-02-11 18:54:40,2013-04-02 21:54:00,0,0,1,0,white
1,953822339,2014-10-28,-monthly-32-blitz-masters-443681,1,2014-10-28 19:03:07,180+2,won on time,A00,lorcho,TomsKantans,David Larino,Toms Kantans,2632,2234,1.0,0.0,17,47,2,28,GM,GM,ESP,LVA,5.0,0.0,16.5,0.00,2013-04-02 21:54:00,2009-02-11 18:54:40,0,0,0,1,white
2,953822340,2014-10-28,-monthly-32-blitz-masters-443681,1,2014-10-28 19:06:45,180+2,won by checkmate,A07,GeorgMeier,marabu44,Georg Meier,,2524,2223,1.0,0.0,4,33,3,29,GM,NM,URY,AND,6.5,3.0,33.0,7.25,2013-01-08 08:11:18,2011-07-29 04:26:11,0,0,1,1,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300121,97374000239,2023-12-26,late-titled-tuesday-blitz-december-26-2023-447...,11,2023-12-27 00:03:31,180+1,won by resignation,B27,morte-negra,miraitrunks,Ricardo Desideri,Arthur Olinto de Souza,2009,2068,1.0,0.0,182,201,327,317,NM,NM,BRA,BRA,3.0,2.0,34.5,29.00,2014-12-29 22:23:49,2017-01-21 14:10:58,0,0,1,1,black
300122,97374000249,2023-12-26,late-titled-tuesday-blitz-december-26-2023-447...,11,2023-12-27 00:03:04,180+1,won by resignation,D02,SmirnovTimofey2000,C_K_G,Тимофей Смирнов,Chanchal Kumer Ghosh,1981,2073,0.0,1.0,213,193,325,312,FM,CM,RUS,BGD,1.0,3.0,31.5,13.50,2015-01-12 21:20:36,2013-08-13 07:23:03,0,0,1,1,black
300122,97374000249,2023-12-26,late-titled-tuesday-blitz-december-26-2023-447...,11,2023-12-27 00:03:04,180+1,won by resignation,D02,C_K_G,SmirnovTimofey2000,Chanchal Kumer Ghosh,Тимофей Смирнов,2073,1981,1.0,0.0,193,213,312,325,CM,FM,BGD,RUS,3.0,1.0,13.5,31.50,2013-08-13 07:23:03,2015-01-12 21:20:36,0,0,1,1,black
300123,97374000255,2023-12-26,late-titled-tuesday-blitz-december-26-2023-447...,11,2023-12-26 23:59:10,180+1,won on time,B00,maurovargas01,protectiago,mauro vargas,Tiago Batista Oliveira,1854,2027,1.0,0.0,189,208,334,320,FM,NM,COL,BRA,3.0,1.5,28.5,35.50,2012-10-11 03:53:55,2018-09-07 14:24:18,0,0,1,1,black


To simplify further data aggregations let's unroll each row (game) of original dataset into 2 (for each side POV).

## Process Data

In [ ]:
# Step 1: Create two versions of the DataFrame `df` with different color assignments
white_df = df.assign(color='white').rename(columns=lambda x: x.replace('white_', '').replace('black_', 'opponent_'))
black_df = df.assign(color='black').rename(columns=lambda x: x.replace('black_', '').replace('white_', 'opponent_'))

# Step 2: Concatenate the two DataFrames vertically
concatenated_df = pd.concat((white_df, black_df))

# Step 3: Sort the concatenated DataFrame by 'game_id' and 'color'
sorted_df = concatenated_df.sort_values(['game_id', 'color'], ascending=[True, False])

# Step 4: Drop columns 'tcn_moves' and 'move_timestamps' from the sorted DataFrame
final_df = sorted_df.drop(columns=['tcn_moves', 'move_timestamps'])

# Assign the result back to `df` if needed
df = final_df



KeyError: "['tcn_moves', 'move_timestamps'] not found in axis"

We'll be using for our analysis games played in 2022-2023 years that include 'early' and 'late' tournaments.

In [ ]:
df = df.query('date.dt.year >= 2022').reset_index(drop=True)

Adding a few more columns that we'll need further. Those include `rating_before`, as chess.com provides rating after the game finished, to avoid any [weird effects](https://reddit.com/r/chess/comments/180q4rs/comment/kad15b6), we'll be using player's rating from previous round.

In [ ]:
df['tournament_type'] = df['tournament_id'].map(lambda x: 'early' if 'early' in x else 'late' if 'late' in x else 'n/a')
df['rating_before'] = df.groupby(['tournament_id', 'handle'])['rating'].shift(1)
df['opponent_rating_before'] = df.groupby(['tournament_id', 'opponent_handle'])['opponent_rating'].shift(1)
df['running_score'] = df.groupby(['tournament_id', 'handle'])['result'].cumsum()
df['opponent_running_score'] = df.eval('result = 1 - result').groupby(['tournament_id', 'opponent_handle'])['result'].cumsum()
df['running_score_before'] = df.groupby(['tournament_id', 'handle'])['running_score'].shift(1, fill_value=0)
df['opponent_running_score_before'] = df.groupby(['tournament_id', 'opponent_handle'])['opponent_running_score'].shift(1, fill_value=0)
df['in_prizes'] = df.eval('final_rank <= 5').astype('int')

# FIT = "finished in top" (I'll explain further)
n_fit = 30
df[f'fit{n_fit}'] = df.eval(f'final_rank <= {n_fit}')

In [ ]:
print('Number of tournaments:', df['tournament_id'].nunique())
print('Number of games:', df['game_id'].nunique())

Number of tournaments: 380
Number of games: 655875


## Analyze Data: 2022-2023 top player stats

For each player we'll calculate number of tournaments/games played, mean/median place and number of times he finished in one of top places (1-10).

In [ ]:
df_top = pd.get_dummies(df.drop_duplicates(['name', 'handle', 'tournament_id']) \
    .query('final_rank <= 10'), columns=['final_rank']) \
    .set_index(['handle', 'name']) \
    .filter(like='final_rank_') \
    .groupby(level=['handle', 'name']).sum() \
    .rename(columns=lambda x: x.replace('final_rank', 'place')) \
    .reset_index()

df_top = pd.merge(
    df.drop_duplicates(['name', 'handle', 'tournament_id'])
      .groupby(['name', 'handle'])
      .agg(place_mean=('final_rank', 'mean'), place_median=('final_rank', 'median'), num_fit=(f'fit{n_fit}', 'sum'))
      .round().astype('int').sort_values('place_median').reset_index(),
    df_top, on=['handle', 'name'], how='outer')
df_top = pd.merge(df.groupby(['handle', 'name'], as_index=False)
                    .agg(
                      total_games=('game_id', 'nunique'),
                      total_tournaments=('tournament_id', 'nunique'),
                      rating_mean=('rating', 'mean'),
                    ),
                  df_top, on=['handle', 'name'], how='outer')

df_top = df_top.fillna(0).sort_values([f'place_{i}' for i in range(1, 11)] + ['rating_mean'], ascending=False)
df_top = df_top.astype({f'place_{i}': 'int' for i in range(1, 11)} | {'rating_mean': 'int'} )
df_top.head(50).style \
    .background_gradient(cmap='Reds', subset=[f'place_{i}' for i in range(1, 11)] + ['rating_mean', 'total_games', 'total_tournaments', 'num_fit']) \
    .background_gradient(cmap='Reds_r', subset=['place_mean', 'place_median'])

,handle,name,total_games,total_tournaments,rating_mean,place_mean,place_median,num_fit,place_1,place_2,place_3,place_4,place_5,place_6,place_7,place_8,place_9,place_10
1863,Hikaru,Hikaru Nakamura,2646,261,3177,38,4,222,76,31,17,19,5,10,10,4,3,5
1414,FairChess_on_YouTube,Dmitry Andreikin,3027,292,3014,27,11,252,25,20,19,14,13,12,16,6,13,7
2644,MagnusCarlsen,Magnus Carlsen,515,50,3235,36,4,44,14,8,3,4,2,1,1,2,2,0
5878,jefferyx,Jeffery Xiong,1746,191,2955,151,26,98,12,12,11,10,4,3,7,4,1,0
1465,Firouzja2003,Alireza Firouzja,1177,131,3081,123,20,71,11,10,8,4,7,6,1,1,0,0
1229,Duhless,Daniil Dubov,1362,145,3027,106,16,88,11,8,10,3,6,7,4,4,2,6
2159,Jospem,Jose Eduardo Martinez Alcantara,2676,268,2983,94,18,168,10,11,5,14,11,12,8,7,7,3
2579,LyonBeast,Maxime Vachier-Lagrave,581,77,2986,197,37,38,9,2,6,4,4,1,3,1,0,0
3171,Oleksandr_Bortnyk,Oleksandr Bortnyk,2675,261,2988,60,16,178,8,10,13,19,7,8,11,4,7,6
7314,wonderfultime,Tuan Minh Le,2038,208,2917,63,16,139,8,8,4,5,9,11,5,8,9,13


Kramnik didn't mention which method he used to estimate performance rating $R_p$, but according to [https://en.wikipedia.org/wiki/Performance_rating_(chess)](https://en.wikipedia.org/wiki/Performance_rating_(chess)) there a few standard methods which we can try:

1. Binary Search
2. Linear



In [ ]:
def performance_rating_bin_search(opponent_ratings, score):

    def expected_score(opponent_ratings, my_rating):
        return (1 / (1 + 10 ** ((opponent_ratings - my_rating) / 400))).sum()

    lo, hi = 1500, 3500

    while hi - lo > 0.001:
        mid = (lo + hi) / 2

        if expected_score(opponent_ratings, mid) < score:
            lo = mid
        else:
            hi = mid

    return round(mid)

def performance_rating_linear(opponent_ratings, score):
    if len(opponent_ratings) > 0:
        p = score / len(opponent_ratings)
        rc = opponent_ratings.mean()
        dp = 800 * p - 400
        rp = rc + dp
        return round(rp)
    return None

# we'll use bin search method further
performance_rating = performance_rating_bin_search

Next we calculate performance rating per player per round (over all tournaments in 2022-2023). We do that for all tournaments and also separately for tournaments where player finishes in top 30 and doesn't.

Here we deviate from Kramnik's method: instead of a bit vague term "fighting for prizes" that was not defined precisely, we'll be using a more simple criteria - whether or not a player has finished in top 30. We may use any number of top places which seems more reasonable, but I took 30, to have more samples and players, as if we restrict it too strongly (e.g. top 10 or 5 places), we might get too few tournaments for some players to estimate their performance reliably.

I explained in detail reasons why I switched to this criteria from "fighting for prizes" in my another my notebook
[What's wrong with Kramnik's "fighting for prizes" notion](https://www.kaggle.com/code/stassl/what-s-wrong-with-fighting-for-prizes-notion)

_**FIT = "finished in top"**_

In [ ]:
def calc_rp(g):
    g_fit = g.query(f'fit{n_fit} == 1')
    g_not_fit = g.query(f'fit{n_fit} == 0')
    return pd.Series({
        'round_rp': performance_rating(g['opponent_rating_before'].values, g['result'].sum()),
        'round_rp_fit': performance_rating(g_fit['opponent_rating_before'].values, g_fit['result'].sum()),
        'round_rp_not_fit': performance_rating(g_not_fit['opponent_rating_before'].values, g_not_fit['result'].sum()),
        'num_total': len(g),
        'num_fit': len(g_fit),
        'num_not_fit': len(g_not_fit)
    })

fit_ever = df_top.query('num_fit > 0').handle
df_round_rp = df.query('handle in @fit_ever and opponent_rating_before.notna()') \
    .groupby(['handle', 'round']) \
    .parallel_apply(calc_rp) \
    .reset_index() \
    .sort_values(['round', 'handle'])

df_round_rp

In [ ]:
print('Players number FIT at least once:', df_round_rp['handle'].nunique())
print('Players number FIT at least 20 times:', df_round_rp.query('num_fit >= 20')['handle'].nunique())

We have **814** players who finished in top at least once, but only for **69** we have enough data to estimate their performance more or less reliably.

Now let's calculate average performance rating for each player for rounds 7-10 and calculate difference in performance in comparison with 11th round.

Another important difference from Kramnik's method, is that when computing the difference I take performance in rounds 7-10 in the same tournaments as for 11th round (when they FIT), to make things comparable.

In [ ]:
df_compare = pd.merge(df_round_rp.query('7 <= round <= 10').groupby('handle')
                 [['round_rp', 'round_rp_fit', 'round_rp_not_fit']].mean(),
               df_round_rp.query('round == 11')
                 [['round_rp', 'round_rp_fit', 'round_rp_not_fit', 'handle']],
               on='handle', suffixes=('_r7-10', '_r11'))

df_compare = pd.merge(df_compare, df_round_rp.drop_duplicates('handle', keep='last')[['handle', 'num_total', 'num_fit']])
df_compare.insert(0, 'delta 11 vs 7-10 when finished in top', df_compare.eval('round_rp_fit_r11 - `round_rp_fit_r7-10`'))

df_compare = df_compare.sort_values('delta 11 vs 7-10 when finished in top', ascending=False).set_index('handle').round() \
    .rename(columns=lambda x: x.replace('round_', '').replace('_', ' '))

Here is plot with all **814** players where on X axis we have number of tournaments used to estimate their performance when they finished in top, on Y axis there is delta between 11 and 7-10 rounds. This basically just extended version of the table that Kramnik presented, but it includes more players, even if they had too few tournaments.

As you can see there is quite large dispersion in the beginning of plot because of small number of tournaments, but as this number increases we get more accurate results and after ~20 tournaments it stays within more or less narrow bounds around 0.

In [ ]:
df_plot = df_compare.assign(delta=lambda x: x['delta 11 vs 7-10 when finished in top'].map(lambda x: '+' if x > 0 else '-'))
(
  so.Plot(df_plot, x='num fit', y='delta 11 vs 7-10 when finished in top', color='delta')
    .add(so.Line(color='gray', linestyle='--'), x=20, y=[10000, -10000], color=None, data={})
    .add(so.Dot(pointsize=2), so.Jitter(x=1.5))
    .limit(y=(-2000, 2000), x=(-1, None))
    .scale(color=['C3', 'C0'], x=so.Continuous().tick(every=10))
    .layout(size=(10, 5))
    .label(title=f'Delta performance of player in rounds 11 and 7-10 when finished in top {n_fit}\n(each dot is a player)]',
           x='Number of times player was in top 30', y='Delta', color='Delta')
)

Here is table of players right to the dashed line, who had more than 20 tournaments (FIT):

In [ ]:
df_compare.query('`num fit` >= 20') \
    .style.bar(subset=['delta 11 vs 7-10 when finished in top'], align='mid', width=80, color=['#4d6ecd', '#db4a4b']) \
    .format(precision=0).set_table_styles([{'selector': 'th', 'props': [('max-width', '120px')]}]) \
    .set_properties(subset=['rp fit r7-10', 'rp fit r11'], **{'background-color': '#fffc6b'})

Here we have only 8 players out of 69 who overperformed by more that 100 points, though you should consider number of tournaments, as if this number is too low, it might be less accurate. This looks not as bad as 11 out of 18 in Kramnik's table.

Kramnik and Firouzja are underperforming quite significantly, which aligns with Kramnik's results, despite our methods have a few differences.

If you think that the criteria "finished in top 30" is not strong enough to differentiate cheaters, you might try lower values. Though if you change it to, let say, 15, you'll get approximately 2x less players and 2x less tournaments for each of them, and performance estimates will be more noisy, but still I would say results will look more or less similar, though a bit shuffled.

Let's plot how individual performance ratings change over rounds for all players.

In [ ]:
df_plot = pd.merge(df_round_rp.query('num_fit >= 20').melt(id_vars=['handle', 'round'],
         value_vars=['round_rp', 'round_rp_fit', 'round_rp_not_fit'],
         value_name='rp', var_name='cohort')
            .assign(cohort=lambda x: x['cohort'].str.replace('round_rp', 'all').str.replace('all_', '')),
         df_round_rp.query('num_fit >= 20').melt(id_vars=['handle', 'round'],
         value_vars=['num_total', 'num_fit', 'num_not_fit'],
         value_name='n', var_name='cohort')
            .assign(cohort=lambda x: x['cohort'].str.replace('num_total', 'all').str.replace('num_', '')),
         on=['handle', 'round', 'cohort']
        ) \
    .join(df.groupby('handle').agg(rating_mean=('rating', 'mean')).round(), on='handle')

(
    so.Plot(df_plot, x='round', y='rp', color='cohort')
        .add(so.Line(linewidth=1, color='C2', linestyle='--', alpha=0.4), y='rating_mean', label='average')
        .add(so.Line())
        .facet(col='handle', wrap=4, order=df_compare.query('`num fit` >= 20').index)
        .scale(x=so.Nominal(), y=so.Continuous().tick(every=200), color=['grey', 'C3', 'C0'])
        .share(x=False)
        .limit(y=(2000, 3500))
        .layout(size=(10, 40))
        .label(title=lambda x: f'{x}\n({df_compare.loc[x, "delta 11 vs 7-10 when finished in top"]:+.0f}/{df_compare.loc[x, "num fit"]})', x='Round', y='Performance Rating')
)

*1st number in brackets is performance rating delta (as in table), 2nd - number of tournaments (FIT)*

*Green dashed line show actual player's chess.com blitz rating averaged over 2022-2023 years*

## Final thoughts

Although I changed a few details, which hopefully leads to more accurate results, but the general idea of comparing performance of players in different rounds belongs to Kramnik, and I think it is quite interesting. Whether or not it could detect potential cheaters is a question. Some people have impressively flat performance lines from start to finish, others have some variation in 11th round (not only) that besides cheating could be explained by how people react to stress and pressure.

Another drawback of this method is that it is applied only to a small portion of all players - 69 out of more than 4K total titled players, to those who finished in top places, but I guess there could be much more cheating among lower rated players, who are not competing for prizes, who might do it occasionally just for fun. And this method is based on quite specific assumption that player will cheat only in a single last round, though if he cheats in previous rounds as well, there will be no difference in performance between earlier rounds and the last one, thus despite someone cheated even more, the method will not detect them.